In [ ]:
%matplotlib inline
#import nbs.utils; reload(nbs/utils)
#from utils import *
import matplotlib.pyplot as plt
import math,sys,os
import numpy as np
import numpy.random as rand
import tempfile
from matplotlib import rcParams, animation, rc
from __future__ import print_function, division
from ipywidgets import interact, interactive, fixed
from ipywidgets.widgets import *
rc('animation', html='html5')
rcParams['figure.figsize'] = 3, 3
%precision 4
np.set_printoptions(precision=4, linewidth=100)


from ahkab import new_ac, run, new_op, new_pz, new_tran, time_functions
from ahkab.circuit import Circuit
from ahkab.plotting import plot_results # calls matplotlib for you
from ahkab.netlist_parser import parse_circuit, parse_analysis

import ahkab

### Small-signal transfer function of various opamp configurations

![RLC Filter](./pictures/leaky_integrator.png)

In [ ]:
(circ, directives, postproc_direct) = parse_circuit('ckts/leaky_integrator.ckt')
print("number of simulations: " + str(len(ahkab.netlist_parser.parse_analysis(circ, directives))))

an = parse_analysis(circ, directives)
for a in an:
  a['outfile']=tempfile.mktemp('')

print("Circuit description:")
print(circ)
print("\nTests:")
print(directives)


In [ ]:
res = run(circ, an)
print("#################################################\nResults: ")
print(res['op'])
print("#####################\n")

for x in res['symbolic']:
  print(x)

In [ ]:
tran = ahkab.new_tran(tstart=0., tstop=7.5e-9, tstep=.01e-9, method='trap')
r = run(circ, tran)['tran']

plt.figure(figsize=[12,8],dpi=100)
# plt.title(cir.title + " - TRAN Simulation")
plt.plot(r['T'], r['VIN'], label="Input voltage")
plt.plot(r['T'], r['VINV'], label="VINV voltage")
plt.plot(r['T'], -r['VOUT'], label="output voltage(inverted)")
plt.legend()
plt.grid(True)
# plt.ylim([0,Vin*1.5])
plt.ylabel('Step response')
plt.xlabel('Time [s]')
